In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 KB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.9/610.9 KB 44.4 MB/s eta 0:00:00
  Created wheel for ibm_db: filename=ibm_db-3.1.0-cp39-cp39-linux_x86_64.whl size=41725861 sha256=de000fafe2be0d93ca86debb0d6982a2f77aa63da09d896e4032f7a31f7cf86e
  Stored in directory: /root/.cache/pip/wheels/0a/dd/10/4a9ad59949e39786d729813e4bce24ccf2263c6d60a62de2f2
  Created wheel for ibm_db_sa: filename=ibm_db_sa-0.3.3-py3-none-any.whl size=27427 sha256=ad791eacee93d1d2d57cfb48a95cebd78f6ecec139340f8909a66bf594289a8e
  Stored in dir

In [ ]:
import ibm_db
import pandas as pd
import ibm_db_dbi

In [ ]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid =        # e.g. "abc12345"
dsn_pwd =    # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port =                 # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=1bbf73c5-d84a-4bb0-85b9-ab1a4348f4a4.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud;PORT=32286;PROTOCOL=TCPIP;UID=zfn82944;PWD=3xGddkWKHzA5XBE8;SECURITY=SSL;


In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  zfn82944 on host:  1bbf73c5-d84a-4bb0-85b9-ab1a4348f4a4.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud


In [ ]:
pconn = ibm_db_dbi.Connection(conn)

In [ ]:
# Exercise 1: Connect to your database server and print its version

server = ibm_db.server_info(conn)
print("DBMS VERSION: ",server.DBMS_VER)

DBMS VERSION:  11.05.0800


In [ ]:
# Exercise 2: Fetch Hospital and Doctor Information using hospital Id and doctor Id
docId = input("Write the doctor Id: ")
hosId = input("Write the hospital Id: ")

Write the doctor Id: 1
Write the hospital Id: 101


In [ ]:
createQuery = "SELECT d.DOCID, d.DOCNAME, d.HOSPITALID, h.NAME, h.BEDCOUNT \
FROM DOCTOR d \
JOIN HOSPITAL h ON d.HOSPITALID=h.ID \
WHERE d.DOCID={0} AND h.ID={1};".format(docId, hosId)
Data = pd.read_sql(createQuery, pconn)
Data

,DOCID,DOCNAME,HOSPITALID,NAME,BEDCOUNT
0,1,Michael,101,Mayo Clinic,230


In [ ]:
# Exercise 3: Get the list Of doctors as per the given specialty and salary
speciality = input("Write the speciality: ")
salary = input("Salary should be greater than :")
createQuery = "SELECT * FROM DOCTOR \
WHERE SPECIALITY='{0}' AND SALARY>={1};".format(speciality, salary)
Data = pd.read_sql(createQuery, pconn)
Data

Write the speciality: Gyane
Salary should be greater than :40000


,DOCID,DOCNAME,HOSPITALID,JOININGDATE,SPECIALITY,EXP,SALARY
0,2,Linda,101,2007-08-08,Gyane,5,850000.0
1,6,Robert,104,1998-09-04,Gyane,14,940000.0


In [ ]:
# Exercise 4: Get a list of doctors from a given hospital
hospital = input("Indicate hospital Id: ")
createQuery = "SELECT D.DOCID,	D.DOCNAME,	D.JOININGDATE,	D.SPECIALITY,	D.EXP,	D.SALARY FROM DOCTOR D LEFT JOIN HOSPITAL H ON D.HOSPITALID = H.ID WHERE H.ID = {0}".format(hospital)
Data = pd.read_sql(createQuery, pconn)
Data

Indicate hospital Id: 101


,DOCID,DOCNAME,JOININGDATE,SPECIALITY,EXP,SALARY
0,1,Michael,2005-02-10,Pediatric,10,900000.0
1,2,Linda,2007-08-08,Gyane,5,850000.0
2,4,Richard,2011-09-05,Pediatric,12,950000.0


In [ ]:
# Exercise 5: Update doctor experience in years
docId = input("Write the doctor Id: ")
exp = input('Write doctor experience in Years: ')
createQuery = "UPDATE DOCTOR SET EXP={0} WHERE DOCID={1};".format(exp, docId)
stmt = ibm_db.exec_immediate(conn ,createQuery)

Write the doctor Id: 1
Write doctor experience in Years: 20


In [ ]:
createQuery = "SELECT * FROM DOCTOR"
Data = pd.read_sql(createQuery, pconn)
Data

,DOCID,DOCNAME,HOSPITALID,JOININGDATE,SPECIALITY,EXP,SALARY
0,1,Michael,101,2005-02-10,Pediatric,20,900000.0
1,2,Linda,101,2007-08-08,Gyane,5,850000.0
2,3,William,102,2004-09-11,Cardialogist,1,1000000.0
3,4,Richard,101,2011-09-05,Pediatric,12,950000.0
4,5,Karen,103,2020-09-05,Oncologist,15,1100000.0
5,6,Robert,104,1998-09-04,Gyane,14,940000.0
6,7,Susan,105,1994-06-05,Oncologist,11,870000.0
7,8,Nancy,106,1994-06-05,Cardialogist,14,870000.0
8,9,Nick,107,2019-06-05,Cardialogist,9,770000.0
